In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
import os

Using TensorFlow backend.


In [2]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 60 # 訓練的 epochs 數量

# 讀取資料並檢視
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# 對 label 進行 one-hot encoding (y_trian 原本是純數字)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


## 首先我們使用一般的 DNN (MLP) 來訓練
由於 DNN 只能輸入一維的資料，我們要先將影像進行攤平，若 (50000, 32, 32, 3) 的影像，攤平後會變成 (50000, 32*32*3) = (50000, 3072)

In [3]:
# 將資料攤平成一維資料
x_train = x_train.reshape(50000, 3072) 
x_test = x_test.reshape(10000, 3072)

# 將資料變為 float32 並標準化
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

50000 train samples
10000 test samples


In [4]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(3072,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

W0805 14:13:35.097588  6788 deprecation_wrapper.py:119] From D:\Anaconda3\envs\keras37\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0805 14:13:35.110323  6788 deprecation_wrapper.py:119] From D:\Anaconda3\envs\keras37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0805 14:13:35.110323  6788 deprecation_wrapper.py:119] From D:\Anaconda3\envs\keras37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0805 14:13:35.119408  6788 deprecation_wrapper.py:119] From D:\Anaconda3\envs\keras37\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0805 14:13:35.123410  6788 deprecation.py

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               1573376   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 1,841,162
Trainable params: 1,841,162
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/60
50000/50000 [==============================] - 3s 55us/step - loss: 2.2

50000/50000 [==============================] - 2s 31us/step - loss: 1.4311 - acc: 0.4932 - val_loss: 1.4891 - val_acc: 0.4759
Epoch 53/60
50000/50000 [==============================] - 2s 31us/step - loss: 1.4357 - acc: 0.4917 - val_loss: 1.4449 - val_acc: 0.4974
Epoch 54/60
50000/50000 [==============================] - 2s 30us/step - loss: 1.4374 - acc: 0.4925 - val_loss: 1.4834 - val_acc: 0.4826
Epoch 55/60
50000/50000 [==============================] - 2s 30us/step - loss: 1.4306 - acc: 0.4942 - val_loss: 1.4560 - val_acc: 0.4853
Epoch 56/60
50000/50000 [==============================] - 2s 31us/step - loss: 1.4324 - acc: 0.4940 - val_loss: 1.4579 - val_acc: 0.4903
Epoch 57/60
50000/50000 [==============================] - 2s 31us/step - loss: 1.4316 - acc: 0.4918 - val_loss: 1.4844 - val_acc: 0.4790
Epoch 58/60
50000/50000 [==============================] - 2s 31us/step - loss: 1.4294 - acc: 0.4969 - val_loss: 1.4669 - val_acc: 0.4727
Epoch 59/60
50000/50000 [=====================

## 接下來我們使用 CNN 來訓練神經網路
CNN 的原理非常適合處理影像類的資料，就讓我們來看看，同樣的訓練條件，CNN 是否顯著優於 DNN 呢?

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [6]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

W0805 14:15:09.396364  6788 deprecation_wrapper.py:119] From D:\Anaconda3\envs\keras37\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
__________

50000/50000 [==============================] - 4s 85us/step - loss: 0.6389 - acc: 0.7950 - val_loss: 0.7088 - val_acc: 0.7643
Epoch 41/60
50000/50000 [==============================] - 4s 85us/step - loss: 0.6338 - acc: 0.7954 - val_loss: 0.8479 - val_acc: 0.7717
Epoch 42/60
50000/50000 [==============================] - 4s 85us/step - loss: 0.6456 - acc: 0.7933 - val_loss: 0.7551 - val_acc: 0.7841
Epoch 43/60
50000/50000 [==============================] - 4s 85us/step - loss: 0.6362 - acc: 0.7953 - val_loss: 0.7536 - val_acc: 0.7638
Epoch 44/60
50000/50000 [==============================] - 4s 85us/step - loss: 0.6414 - acc: 0.7950 - val_loss: 0.7054 - val_acc: 0.7909
Epoch 45/60
50000/50000 [==============================] - 4s 85us/step - loss: 0.6586 - acc: 0.7894 - val_loss: 0.7676 - val_acc: 0.7620
Epoch 46/60
50000/50000 [==============================] - 4s 85us/step - loss: 0.6562 - acc: 0.7899 - val_loss: 0.8257 - val_acc: 0.7566
Epoch 47/60
50000/50000 [=====================

## 同樣運算 10 個 epochs，但 CNN 在 test data 的準確率顯著優於 DNN!

## 作業
1. 請試著調整各個超參數，並說明那些超參數對於結果有明顯的影響?

   [ANS] Epochs的數量對於結果影響較大，當epochs=50時，可以獲得最高0.7818的accuracy
   
   (1). Batch_size = 64, epochs = 10
        DNN loss: 1.8032261348724365 / accuracy: 0.3443
        CNN loss: 1.1346897183418274 / accuracy: 0.6682
   
   (2). Batch_size = 128, epochs = 10
        DNN loss: 1.6256148872375489 / accuracy: 0.4162
        CNN loss: 0.6931948656082153 / accuracy: 0.7642
   
   (3). Batch_size = 256, epochs = 10
        DNN loss: 1.700641077041626 / accuracy: 0.3988
        CNN loss: 0.766863065624237 / accuracy: 0.7344
   
   (4). Batch_size = 512, epochs = 10
        DNN loss: 1.679827537727356 / accuracy: 0.4037
        CNN loss: 0.8638354285240173 / accuracy: 0.6986
   
   (5). Batch_size = 128, epochs = 50
        DNN loss: 1.4592716119766236 / accuracy: 0.4907
        CNN loss: 0.6870723120212555 / accuracy: 0.7818
   
   (6). Batch_size = 128, epochs = 100
        DNN loss: 1.530925919342041 / accuracy: 0.4711
        CNN loss: 0.7844122951984406 / accuracy: 0.7555

2. CNN 與 DNN 哪個模型的參數數量比較多? 造成參數的數量不同的原因在哪?
   
   [ANS] CNN 有 Filter 和 MaxPooling2D 可以只保留較重要的特徵，因此特徵維度會降低，參數也會比 DNN 來的少


   